Scraping Job Postings from LinkedIn 

Resource used to develop code: 
https://maoviola.medium.com/a-complete-guide-to-web-scraping-linkedin-job-postings-ad290fcaa97f

In [1]:
#Import packages
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup

## importing libraries to plot the wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Hide Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# URL for LinkedIn Job search of jobs in Rancho Cordova with 10 mile radius. 
# Past Month
# (Not signed-in search)

url = 'https://www.linkedin.com/jobs/search?keywords=&location=Rancho%20Cordova%2C%20California%2C%20United%20States&locationId=&geoId=104863738&sortBy=R&f_TPR=r2592000&f_PP=104863738&distance=10&position=1&pageNum=0'

In [23]:
#Set-up chromedriver

wd = webdriver.Chrome(executable_path=r'C:\chromedriver.exe')
wd.get(url)

In [24]:
#How many jobs are curently available in Rancho Cordova on LinkedIn

no_of_jobs = int(wd.find_element_by_css_selector('h1>span').get_attribute('innerText'))

print('There are', no_of_jobs, 'jobs available within 10 miles of Rancho Cordova on LinkedIn over the past month.')

There are 941 jobs available within 10 miles of Rancho Cordova on LinkedIn over the past month.


In [25]:
#Browse all jobs for the search.
i = 2
while i <= int(no_of_jobs/25)+1: 
    #Scroll until hit the see more jobs button.
    wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    i = i + 1
    try:
        #Click the see more jobs button and then keep scrolling.
        wd.find_element_by_xpath(f"//button[@aria-label='Load more results']").click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

In [26]:
#Create a list of the jobs.
job_lists = wd.find_element_by_class_name('jobs-search__results-list')
jobs = job_lists.find_elements_by_tag_name('li')

In [27]:
#Test that it collected all jobs.
#Some jobs may be posted or removed during collection, resulting in more or less.
len(jobs)

944

In [18]:
#Pull basic information from each job.
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute('data-id')
    job_id.append(job_id0)
 
    job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)
 
    company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)
 
    date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)
 
    job_link0 = job.find_element_by_css_selector('a').get_attribute('href')
    job_link.append(job_link0)

In [19]:
#See first 10 of company name to verify that it worked correctly.
company_name[:10]

['Kaiser Permanente',
 'Kaiser Permanente',
 'Kaiser Permanente',
 'Kaiser Permanente',
 'Dignity Health',
 'Centene Corporation',
 'Dignity Health',
 'Centene Corporation',
 'Anthem, Inc.',
 'Anthem, Inc.']

In [28]:
jd = []

for job in jobs:
        
    job.click()
    
    jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div'
    jd0 = wd.find_element_by_xpath(jd_path).get_attribute('innerText')
    jd.append(jd0)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FF06F3+2492147]
	Ordinal0 [0x00F89BD1+2071505]
	Ordinal0 [0x00E92478+1057912]
	Ordinal0 [0x00E94BA4+1067940]
	Ordinal0 [0x00E94A6E+1067630]
	Ordinal0 [0x00E9531A+1069850]
	Ordinal0 [0x00EE584B+1398859]
	Ordinal0 [0x00ED57C3+1333187]
	Ordinal0 [0x00EE4F38+1396536]
	Ordinal0 [0x00ED568B+1332875]
	Ordinal0 [0x00EB21D4+1188308]
	Ordinal0 [0x00EB302F+1191983]
	GetHandleVerifier [0x011767A6+1545030]
	GetHandleVerifier [0x0122105C+2243580]
	GetHandleVerifier [0x0107BC97+518199]
	GetHandleVerifier [0x0107AD80+514336]
	Ordinal0 [0x00F8ED2D+2092333]
	Ordinal0 [0x00F92EE8+2109160]
	Ordinal0 [0x00F93022+2109474]
	Ordinal0 [0x00F9CB71+2149233]
	BaseThreadInitThunk [0x7601FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77917A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77917A6E+238]


In [29]:
len(jd)

4

In [14]:
#Pull detailed information from each job.
jd = []
seniority = []
emp_type = []
job_func = []
industries = []

for item in range(len(jobs)):
    job_func0=[]
    industries0=[]

    # clicking job to view job details
    job_click_path = 'li'
    job_click = job.find_element_by_css_selector(job_click_path).click()
    time.sleep(5)
    
    jd_path = '/html/body/main/section/div[2]/section[2]/div'
    jd0 = wd.find_element_by_xpath(jd_path).get_attribute('innerText')
    jd.append(jd0)
    
    seniority_path = f'/html/body/main/section/div[2]/section[2]/ul/li[1]/span'
    seniority0 = wd.find_element_by_xpath(seniority_path).get_attribute('innerText')
    seniority.append(seniority0)
 
    emp_type_path = f'/html/body/main/section/div[2]/section[2]/ul/li[2]/span'
    emp_type0 = wd.find_element_by_xpath(emp_type_path).get_attribute('innerText')
    emp_type.append(emp_type0)
 
    job_func_path = f'/html/body/main/section/div[2]/section[2]/ul/li[3]/span'
    job_func_elements = job.find_elements_by_xpath(job_func_path)
    for element in job_func_elements:
        job_func0.append(element.get_attribute('innerText'))
        job_func_final = ', '.join(job_func0)
        job_func.append(job_func_final)
                    
    industries_path = f'/html/body/main/section/div[2]/section[2]/ul/li[4]/span'
    industries_elements = job.find_elements_by_xpath(industries_path)
    for element in industries_elements:
        industries0.append(element.get_attribute('innerText'))
        industries_final = ', '.join(industries0)
        industries.append(industries_final)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"li"}
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00FF06F3+2492147]
	Ordinal0 [0x00F89BD1+2071505]
	Ordinal0 [0x00E92478+1057912]
	Ordinal0 [0x00EBC964+1231204]
	Ordinal0 [0x00EB34E1+1193185]
	Ordinal0 [0x00ED57FA+1333242]
	Ordinal0 [0x00EB3466+1193062]
	Ordinal0 [0x00ED58AA+1333418]
	Ordinal0 [0x00EE4F38+1396536]
	Ordinal0 [0x00ED568B+1332875]
	Ordinal0 [0x00EB21D4+1188308]
	Ordinal0 [0x00EB302F+1191983]
	GetHandleVerifier [0x011767A6+1545030]
	GetHandleVerifier [0x0122105C+2243580]
	GetHandleVerifier [0x0107BC97+518199]
	GetHandleVerifier [0x0107AD80+514336]
	Ordinal0 [0x00F8ED2D+2092333]
	Ordinal0 [0x00F92EE8+2109160]
	Ordinal0 [0x00F93022+2109474]
	Ordinal0 [0x00F9CB71+2149233]
	BaseThreadInitThunk [0x7601FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77917A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77917A6E+238]


In [29]:
jd[:2]

[]